In [84]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
nltk.download('wordnet')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Downloading package wordnet to /home/anasofia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [25]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [57]:
sw = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
text = data.problem_statement

text = text.apply(lambda x: re.sub('[,|.|(|)|$|!|?|!]',' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo
# bigrama e trigrama todo

0       [given, n, rectangles, corners, rectangles, in...
1       [Daniel, organizing, football, tournament, com...
2       [Piegirl, found, monster, book, monsters, pies...
3       [far, away, land, exists, planet, shaped, like...
4       [Piegirl, found, red, button, one, last, chanc...
                              ...                        
8338    [n, blocks, arranged, row, numbered, left, rig...
8339    [map, capital, Berland, viewed, infinite, coor...
8340    [play, strategic, video, game, yeah, ran, good...
8341    [first, let's, define, function, f, x, follows...
8342    [Recently, lot, students, enrolled, Berland, S...
Name: problem_statement, Length: 6819, dtype: object


In [56]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    n block arranged row numbered left right start...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let's define function f x follows: \begi...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

## Vertorizando

In [60]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df)

In [70]:
X = vec.fit_transform(text)
X.shape

(6819, 250)

## Agrupando

In [78]:
n_components = 20 # todo
max_iter = 100

lda = LatentDirichletAllocation(n_components=n_components, max_iter=max_iter)

In [79]:
lda_matrix = lda.fit_transform(X)

In [80]:
top_word = lda.components_
terms = vec.get_feature_names_out()

for i, comp in enumerate(top_word):
    top_terms_key = zip(terms, comp)
    top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
    top_terms_list= list(dict(top_terms_key).keys())
    print("Topic "+str(i)+": ",top_terms_list)

Topic 0:  ['sequence', 'query', 'type', 'answer', 'length', 'one', 'given', 'second', 'following', 'two']
Topic 1:  ['problem', 'output', 'use', 'one', 'program', 'task', 'input', 'write', 'given', 'space']
Topic 2:  ['time', 'digit', 'position', 'moment', 'show', 'start', 'zero', 'greater', 'sum', '10']
Topic 3:  ['day', 'segment', 'friend', 'th', 'one', 'end', 'call', 'exactly', 'next', 'consecutive']
Topic 4:  ['point', 'coordinate', 'xi', 'distance', 'two', 'th', 'answer', 'located', 'absolute', 'direction']
Topic 5:  ['operation', 'change', 'perform', 'following', 'make', 'minimum', 'equal', 'sequence', 'one', 'initial']
Topic 6:  ['leq', '10', 'ldots', 'a_i', 'th', 'example', 'two', 'cdot', 'second', 'next']
Topic 7:  ['edge', 'graph', 'path', 'vertex', 'connected', 'two', 'given', 'pair', 'contain', 'one']
Topic 8:  ['tree', 'vertex', 'edge', 'two', 'connected', 'given', 'next', 'guaranteed', 'one', 'distance']
Topic 9:  ['print', 'without', 'otherwise', 'yes', 'quote', 'two', '

## Validando

In [ ]:
# todo

## Visualizando

In [85]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda, X, vec, mds='tsne')
panel
# todo

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'